In [7]:
from converter import CoNLLConverter

Это самописный преобразователь датасета в нужный формат. Датасет CoNLL - это тексты со списокм ошибок. В данном случае, нам нужны всего лишь исходный текст и исправленный текст

In [8]:
converter = CoNLLConverter()
converter.prepare('conll14st-test-data/noalt/official-2014.0.sgml', 'source.txt', 'target.txt')

Теперь исходный датасет лежит в source.txt, а исправленный - в target.txt

Далее предполагаем, что рядом лежит склонированный репозиторий https://github.com/grammarly/gector.

Теперь датасет нужно преобразовать в ещё один формат. Задача GEC сводится к теггированию токенов с конечным набором тегов. Тег задаёт, какому исправлению должен подвергнуться данный токен (тег $KEEP, если токен остаётся без изменений).</br>
Я детально изучил преобразователь в gector. Не счёл нужным писать свой, т.к. выйдет коряво и, точно, с меньшим числом поддерживаемых тегов. Да и зачем, если есть готовый.</br>


Если вкратце, идея такая. Сравнивается исходый текст и исправленный. Дл начала, используется difflib.SequenceMatcher.get_opcodes(). Результаты 'equal', 'insert', 'delete' становятся соответствующими метками. При этом 'insert' предполагает некоторое конечное множество возможных вставляемых слов (т.е.множество меток конечно), остальные метки независимы от токенов. Если get_opcodes возвращает 'replace', то дальше начинается длинный анализ, что это конкретно может быть за ошибка такая, и в зависимости от этого выставляется метка. В крайнем случае, может быть поставлена и просто метка $REPLACE. Согласно статье, имеется 4971 меток: "basic transformations (token-independent KEEP, DELETE and 1167 token-dependent APPEND, 3802 REPLACE) and 29 token-independent gtransformations."

Применяем, наконец, вышеупомянутый преобразователь датасета.

In [9]:
!python3 gector/utils/preprocess_data.py -s source.txt -t target.txt -o dataset.txt

The size of raw dataset is 252
252it [00:00, 827.48it/s]
Overall extracted 252. Original TP 237. Original TN 15


Ридер тоже возьмём из gector. Это логично: если преобразовали данные в текущий формат инструментом из gector, то и извлеь обратно надёжнее всего с помощью него же.</br>
Только придётся подредактировал пути в импортах gector, т.к. там используются относительный пути:
в datareader необходимо utils.helper исправить на gector.utils.helper

In [11]:
from gector.gector.datareader import Seq2LabelsDatasetReader

Модель возьмём из pos-tagging, только подредактируем аргументы forward, чтобы не переписывать datareader.

In [12]:
from model import POSTagger

In [13]:
import torch
from allennlp.data.vocabulary import Vocabulary
from allennlp.common import Params
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding, ElmoTokenEmbedder, TokenCharactersEncoder
from allennlp.modules.seq2seq_encoders import PytorchSeq2SeqWrapper
from allennlp.modules.seq2vec_encoders import PytorchSeq2VecWrapper
from allennlp.data.iterators import BucketIterator
from allennlp.training.trainer import Trainer
from allennlp.data.token_indexers import ELMoTokenCharactersIndexer

В оригинальной статье в качестве эмбеддера и энкодера использовались много раз предобучаемые и обучаемые БЕРТы. Я ограничился использованием в качестве эмеддера ELMo, т.к. во-первых, в отличие от БЕРТ-а, с ним я никогда раньше не работал, а, во-вторых, ELMo намного быстрее обучается.

In [14]:
reader = Seq2LabelsDatasetReader(token_indexers={'tokens': ELMoTokenCharactersIndexer()}, tn_prob=0, tp_prob=1)
dataset = reader.read('dataset.txt')
vocab = Vocabulary.from_instances(dataset, pretrained_files={'tokens':'data/cc.en.300.vec'}, only_include_pretrained_words=False)

vocab.get_index_to_token_vocabulary('labels')

252it [00:00, 2956.21it/s]
100%|██████████| 252/252 [00:00<00:00, 18422.04it/s]


{0: '$KEEP',
 1: '$DELETE',
 2: '$TRANSFORM_AGREEMENT_PLURAL',
 3: '$TRANSFORM_VERB_VB_VBZ',
 4: '$TRANSFORM_VERB_VBZ_VB',
 5: '$APPEND_the',
 6: '$APPEND_a',
 7: '$REPLACE_a',
 8: '$REPLACE_to',
 9: '$REPLACE_are',
 10: '$TRANSFORM_VERB_VBN_VB',
 11: '$REPLACE_is',
 12: '$TRANSFORM_CASE_CAPITAL',
 13: '$TRANSFORM_VERB_VBG_VB',
 14: '$TRANSFORM_VERB_VB_VBN',
 15: '$REPLACE_the',
 16: '$REPLACE_.',
 17: '$TRANSFORM_CASE_LOWER',
 18: '$TRANSFORM_VERB_VB_VBG',
 19: '$REPLACE_in',
 20: '$REPLACE_of',
 21: '$REPLACE_have',
 22: '$REPLACE_their',
 23: '$APPEND_to',
 24: '$REPLACE_for',
 25: '$REPLACE_and',
 26: '$REPLACE_they',
 27: '$REPLACE_lives.',
 28: '$REPLACE_be',
 29: '$APPEND_are',
 30: '$APPEND_that',
 31: '$TRANSFORM_VERB_VBN_VBZ',
 32: '$REPLACE_can',
 33: '$APPEND_about',
 34: '$REPLACE_with',
 35: '$REPLACE_diseased',
 36: '$TRANSFORM_AGREEMENT_SINGULAR',
 37: '$APPEND_be',
 38: '$REPLACE_about',
 39: '$REPLACE_on',
 40: '$REPLACE_,',
 41: '$REPLACE_were',
 42: '$REPLACE_has',


In [16]:
word_emb = 300
hidden_dim = 300

embedder = BasicTextFieldEmbedder({"tokens": ElmoTokenEmbedder(
                                      options_file='data/options.json',
                                     weight_file='data/weights.hdf5',
                                     projection_dim=word_emb)})
encoder = PytorchSeq2SeqWrapper(torch.nn.LSTM(embedder.get_output_dim(), hidden_dim, batch_first=True, bidirectional=True, num_layers=2))
model = POSTagger(vocab, embedder, encoder)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:434: UserWarning: Setting attributes on ParameterList is not supported.
  warnings.warn("Setting attributes on ParameterList is not supported.")


In [17]:
test_size = len(dataset) // 10
train_dataset = dataset[:-test_size]
dev_dataset = dataset[-test_size:]

А вот эта самая большая проблема: в датасете всего 50 текстов. Даже после разделения их на отдельные строки получается не особо много:

In [18]:
len(dataset)

252

Но попробуем, других данных всё равно нет...

In [19]:
device = torch.device('cuda')
model.to(device)
optimizer = torch.optim.Adam(model.parameters())
iterator = BucketIterator(batch_size=16, sorting_keys=[("tokens", "num_tokens")], biggest_batch_first=True)
iterator.index_with(vocab)
trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=train_dataset,
                  validation_dataset=dev_dataset,
                  patience=5,
                  num_epochs=20,
                  cuda_device=0,
                  validation_metric="+fscore")

In [20]:
trainer.train()

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:434: UserWarning: Setting attributes on ParameterList is not supported.
  warnings.warn("Setting attributes on ParameterList is not supported.")
accuracy: 0.8353, precision: 0.8353, recall: 0.8353, fscore: 0.8353, loss: 5280.9920 ||: 100%|██████████| 15/15 [00:17<00:00,  1.13s/it]
accuracy: 0.8462, precision: 0.8462, recall: 0.8462, fscore: 0.8462, loss: 1695.3635 ||: 100%|██████████| 2/2 [00:01<00:00,  1.48it/s]
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:434: UserWarning: Setting attributes on ParameterList is not supported.
  warnings.warn("Setting attributes on ParameterList is not supported.")
accuracy: 0.8958, precision: 0.8958, recall: 0.8958, fscore: 0.8958, loss: 1717.1580 ||: 100%|██████████| 15/15 [00:16<00:00,  1.09s/it]
accuracy: 0.8462, precision: 0.8462, recall: 0.8462, fscore: 0.8462, loss: 1249.4717 ||: 100%|██████████| 2/2 [00:01<00:00,  1.51it/s]
/usr/local/lib/python3.6/dis

{'best_epoch': 0,
 'best_validation_accuracy': 0.8462316641375822,
 'best_validation_fscore': 0.8462316393852234,
 'best_validation_loss': 1695.3634643554688,
 'best_validation_precision': 0.8462316393852234,
 'best_validation_recall': 0.8462316393852234,
 'epoch': 4,
 'peak_cpu_memory_MB': 4371.6,
 'peak_gpu_0_memory_MB': 14367,
 'training_accuracy': 0.8963682268401632,
 'training_cpu_memory_MB': 4371.6,
 'training_duration': '0:01:30.135396',
 'training_epochs': 4,
 'training_fscore': 0.8963682055473328,
 'training_gpu_0_memory_MB': 14367,
 'training_loss': 1475.0076009114584,
 'training_precision': 0.8963682055473328,
 'training_recall': 0.8963682055473328,
 'training_start_epoch': 0,
 'validation_accuracy': 0.8462316641375822,
 'validation_fscore': 0.8462316393852234,
 'validation_loss': 1229.5848999023438,
 'validation_precision': 0.8462316393852234,
 'validation_recall': 0.8462316393852234}

Посмотрим...

In [21]:
model.eval()
with torch.no_grad():    
    labels =  model.forward_on_instance(dev_dataset[1])['labels']

for token, label in zip(dev_dataset[1]['tokens'].tokens,labels):
    print(token, label)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:434: UserWarning: Setting attributes on ParameterList is not supported.
  warnings.warn("Setting attributes on ParameterList is not supported.")


$START $KEEP
In $KEEP
conclusion, $KEEP
the $KEEP
showing $KEEP
up $KEEP
of $KEEP
social $KEEP
media $KEEP
sites $KEEP
is $KEEP
a $KEEP
double $KEEP
edged $KEEP
sword. $KEEP
It $KEEP
can $KEEP
make $KEEP
our $KEEP
life $KEEP
simpler. $KEEP
People $KEEP
can $KEEP
promote $KEEP
their $KEEP
friendship $KEEP
and $KEEP
relationship $KEEP
through $KEEP
these $KEEP
social $KEEP
media. $KEEP
Whereas, $KEEP
it $KEEP
can $KEEP
also $KEEP
make $KEEP
people $KEEP
forget $KEEP
to $KEEP
communicate $KEEP
to $KEEP
their $KEEP
family $KEEP
and $KEEP
friends $KEEP
in $KEEP
the $KEEP
real $KEEP
life $KEEP
and $KEEP
affect $KEEP
negatively $KEEP
to $KEEP
their $KEEP
interpersonal $KEEP
skills. $KEEP
But $KEEP
overall, $KEEP
the $KEEP
advantages $KEEP
far $KEEP
outweigh $KEEP
its $KEEP
disadvantage. $KEEP
So $KEEP
we $KEEP
need $KEEP
to $KEEP
make $KEEP
best $KEEP
use $KEEP
of $KEEP
these $KEEP
social $KEEP
media $KEEP
and $KEEP
make $KEEP
it $KEEP
a $KEEP
fun $KEEP
way $KEEP
of $KEEP
adding $KEEP
some $K

Ну да, совсем не обучилось. Предлагает все тексты оставить как есть и не мучиться с исправлениями.

Немного подправленная функция из gector. Позволяет получить из исходного текста и предлагаемых исправлений исправленный текст.

In [22]:
from gector.utils.helpers import PAD, UNK, get_target_sent_by_edits, START_TOKEN


def get_token_action(index, sugg_token):
        """Get lost of suggested actions for token."""
        # cases when we don't need to do anything
        if sugg_token in [UNK, PAD, '$KEEP']:
            return None

        if sugg_token.startswith('$REPLACE_') or sugg_token.startswith('$TRANSFORM_') or sugg_token == '$DELETE':
            start_pos = index
            end_pos = index + 1
        elif sugg_token.startswith("$APPEND_") or sugg_token.startswith("$MERGE_"):
            start_pos = index + 1
            end_pos = index + 1

        if sugg_token == "$DELETE":
            sugg_token_clear = ""
        elif sugg_token.startswith('$TRANSFORM_') or sugg_token.startswith("$MERGE_"):
            sugg_token_clear = sugg_token[:]
        else:
            sugg_token_clear = sugg_token[sugg_token.index('_') + 1:]

        return start_pos - 1, end_pos - 1, sugg_token_clear, 0

In [26]:
model.eval()
with torch.no_grad():
    labels = model.forward_on_instances(dev_dataset)

for j in range(len(dev_dataset)):
    tokens = [str(i) for i in dev_dataset[j]['tokens'].tokens]
    actions = []

    for i in range(len(labels[j]['labels'])):
        action = get_token_action(i, labels[j]['labels'][i])
        if action:
            actions.append(action)

    print(' '.join(get_target_sent_by_edits(tokens, actions)[1:]))

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:434: UserWarning: Setting attributes on ParameterList is not supported.
  warnings.warn("Setting attributes on ParameterList is not supported.")


However, social media may do some harm to people's life. It will make people spend more time on chatting on the internet rather than communicate face to face such that the interpersonal skill will be badly affected. Some people spend a lot of time in it and forget their real life. Actually, I have seen some people are very funny and interesting when I am chatting with them online. However, they behave quite boring when I talk to them in real life. Teenagers are developing their interpersonal skills and social skills when they are growing up. If they spend too much time on these social media, it may cause some problems to their mental development. They may feel uncomfortable when they communicate with people face to face. Some may share fewer and fewer time with their family and not knowing how to express themselves. Moreover, some wrong values on the websites will affect teenagers and confuse them. Some may even do something wrong.
In conclusion, the showing up of social media sites is

Как-то так.